# Transfer Learning - VGGNet

**Załadujemy gotowy model (w tym przypadku VGGNet19) i dostroimy go do nowego zadania klasyfikacji binarnej - przykład z książki "Uczenie głębokie i sztuczna inteligencja. Interaktywny przewodnik ilustrowany"**

### Ładujemy zależności

In [16]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Ładujemy wytrenowany model VGG19

In [26]:
vgg19 = VGG19(include_top=False, #ostatnie warstwy zagęszczone, specyficzne dla zbioru, nie zostają załadowane
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

### "Zamrażamy" warstwy modelu VGGNet19 - nie będą się douczać

In [27]:
for layer in vgg19.layers:
    layer.trainable = False

In [30]:
vgg19

### Tworzymy całościowy model, "u góry" dodajemy VGGNet19, a dalej własne warstwy

**Zadanie 3. Dodaj warstwę spłaszczającą, dropout 0.5 i warstwę pozwalającą sklasyfikować binarnie obiekty**

In [45]:
model = Sequential()
model.add(vgg19)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [46]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten_7 (Flatten)         (None, 25088)             0         
                                                                 
 dropout_7 (Dropout)         (None, 25088)             0         
                                                                 
 dense_5 (Dense)             (None, 2)                 50178     
                                                                 
Total params: 20,074,562
Trainable params: 50,178
Non-trainable params: 20,024,384
_________________________________________________________________


### Kompilujemy model

In [47]:
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

### Pobierz zbiór danych z [Kaggle'a](https://www.kaggle.com/datasets/dansbecker/hot-dog-not-hot-dog) i rozpakuj

### Zbiór jest mały, więc tworzymy instancje klasy ImageDataGenerator, które pozwolą nam powiększyć dane

In [48]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

### Deklarujemy rozmiar paczki

In [49]:
batch_size = 32

### Powiększamy dane

In [50]:
train_generator = train_datagen.flow_from_directory(
    directory= 'archive/train',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

valid_generator = valid_datagen.flow_from_directory(
    directory= 'archive/test',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


**Zadanie 4. Wypisz co oznaczają wszystkie zadeklarowane parametry generatorów danych**

<b>directory</b> - ścieżka w której znajdują się foldery klas
<br>
<b>target_size</b>  - rozmiar obrazów wchodzących, tj. zmienione do 224x224
<br>
<b>classess</b>  - dwie klasy daych, foldery w ścieżce zawierające zdjęcia hot dogów lub innych dań
<br>
<b>class_mode</b>  - deklarowany w zależności od ilości klas
<br>
<b>batch_size</b>  - liczba która dokładnie dzieli całkowitą liczbę obrazów
<br>
<b>shuffle</b>  - czy wymieszać kolejność plików
<br>
<b>seed</b>  - kolejny czynnik wpływający na randomowość obrazów poddawanych treningowi/walidacji

### Uczymy model z danymi z generatora (dla ułatwienia używamy do tego oddzielnej funkcji, choć będzie w następnej wersji tensorflowa usunięta)

In [51]:
model.fit_generator(train_generator, steps_per_epoch=15, epochs=16, validation_data=valid_generator, validation_steps=15)

C:\Users\Angelika.Bulkowska\AppData\Local\Temp\ipykernel_35276\2768083069.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=15, epochs=16, validation_data=valid_generator, validation_steps=15)


Epoch 1/16
15/15 [==============================] - 273s 19s/step - loss: 1.2970 - accuracy: 0.5258 - val_loss: 0.6133 - val_accuracy: 0.6750
Epoch 2/16
15/15 [==============================] - ETA: 0s - loss: 0.9253 - accuracy: 0.6266

KeyboardInterrupt: 